In [1]:
# import os
# import pandas as pd
# from deepeval import evaluate
# from deepeval.test_case import LLMTestCase
# from deepeval.metrics import SummarizationMetric
# from deepeval.models import GeminiModel

# def batch_summarization_score(
#     input_output_pairs,
#     api_key,
#     batch_size=100,
#     # sleep_time=60,
#     # max_retries=3,
#     # retry_delay=10
# ):
#     """
#     Evaluates summarization quality in batches using the Gemini model with deepeval,
#     including automatic retries for failed evaluations.

#     Args:
#         input_output_pairs: List of tuples (input, actual_output)
#         api_key: Gemini API key string
#         batch_size: Number of evaluations per batch to avoid hitting rate limits
#         sleep_time: Sleep time in seconds between batches
#         max_retries: Number of retry attempts per batch on failure
#         retry_delay: Delay between retries in seconds

#     Returns:
#         List of dicts containing input, actual_output, score, reason
#     """
#     model = GeminiModel(
#         model_name="gemini-2.0-flash",
#         api_key=api_key,
#         temperature=0,
#     )

#     metric = SummarizationMetric(
#         threshold=0.5,
#         model=model,
#         # assessment_questions=[
#         #     "Is the coverage score based on a percentage of 'yes' answers?",
#         #     "Does the score ensure the summary's accuracy with the source?",
#         #     "Does a higher score mean a more comprehensive summary?"
#         # ]
#         n=10
#     )

#     results = []

#     for i in range(0, len(input_output_pairs), batch_size):
#         batch = input_output_pairs[i:i + batch_size]
#         test_cases = [LLMTestCase(input=inp, actual_output=out) for inp, out in batch]

#         try:
#             batch_result = evaluate(test_cases=test_cases, metrics=[metric])
#             for test_case, res in zip(batch, batch_result.test_results):
#                 results.append(res)
#         except Exception as e:
#             print(f"Batch {i // batch_size + 1} failed with error: {e}")
#             for _ in batch:
#                 results.append(None)  # Preserve alignment with input

#     return results

In [2]:
# import asyncio
# import os
# import pandas as pd
# from deepeval import evaluate
# from deepeval.test_case import LLMTestCase
# from deepeval.metrics import SummarizationMetric
# from deepeval.models import GeminiModel

# async def async_summarization_score(
#     input_output_pairs,
#     api_key,
# ):
#     """
#     Evaluates summarization quality using async individual test case calls with the Gemini model,
#     processing all test cases concurrently with asyncio.gather, returning the metric score or 0 on failure.

#     Args:
#         input_output_pairs: List of tuples (input, actual_output)
#         api_key: Gemini API key string

#     Returns:
#         List of floats containing the summarization score for each test case, or 0 if evaluation fails
#     """
#     model = GeminiModel(
#         model_name="gemini-2.0-flash",
#         api_key=api_key,
#         temperature=0,
#     )

#     metric = SummarizationMetric(
#         threshold=0.5,
#         model=model,
#         n=10
#     )

#     async def evaluate_test_case(test_case):
#         try:
#             result = await evaluate(test_cases=[test_case], metrics=[metric])
#             return result.test_results[0].metrics_data[0].score
#         except Exception as e:
#             print(f"Test case failed with error: {e}")
#             return 0

#     test_cases = [LLMTestCase(input=inp, actual_output=out) for inp, out in input_output_pairs]

#     # Process all test cases concurrently using asyncio.gather
#     results = await asyncio.gather(
#         *[evaluate_test_case(test_case) for test_case in test_cases],
#         return_exceptions=True
#     )

#     # Handle any exceptions returned by gather
#     final_results = [result if isinstance(result, float) else 0 for result in results]

#     return final_results

In [3]:

from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import SummarizationMetric
from deepeval.models import GeminiModel

def summarize_100_pairs(
    input_output_pairs,
    api_key,
    # sleep_time=20,
    # max_retries=1,
    # retry_delay=60
):
    """
    Evaluates summarization quality for 100 pairs using Gemini model and DeepEval.

    Args:
        input_output_pairs: List of 100 (input, output) tuples.
        api_key: Gemini API key.
        sleep_time: Optional pause after evaluation (e.g., for chaining jobs).
        max_retries: Retry attempts per pair.
        retry_delay: Delay between retries.

    Returns:
        List of dicts or evaluation results.
    """
    # if len(input_output_pairs) != 100:
    #     raise ValueError("Expected exactly 100 input-output pairs.")

    model = GeminiModel(
        model_name="gemini-2.0-flash",
        api_key=api_key,
        temperature=0,
    )

    metric = SummarizationMetric(
        threshold=0.5,
        model=model,
        n=10
    )

    results = []

    for idx, (inp, out) in enumerate(input_output_pairs):
        try:
            test_case = LLMTestCase(input=inp, actual_output=out)
            result = evaluate(test_cases=[test_case], metrics=[metric])
            results.append(result.test_results[0].metrics_data[0].score)
        except Exception as e:
            print(f"Error evaluating pair {idx + 1}: {e}")
            results.append(0)
    return results


In [4]:
import os
API_KEY = os.getenv("GEMINI_API_KEY")

In [5]:
import pandas as pd
df = pd.read_csv("/Users/pupipatsingkhorn/Developer/repositories/NanoLLaDA/data/gemini_summaries.csv").sample(10)

In [6]:
scores = summarize_100_pairs(
    input_output_pairs=list(
        df[["body", "generated"]].itertuples(index=False, name=None)
    ),
    api_key=API_KEY,
)

✨ You're running DeepEval's latest Summarization Metric! (using gemini-2.0-flash, strict=False, 
async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:06,  6.09s/test case]



Metrics Summary

  - ✅ Summarization (score: 0.6, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash, reason: The score is 0.60 because the summary omits answers to several questions that the original text could answer, such as details about Manning's fines, a suicide attempt, hospitalization, and beliefs about the grand jury., error: None)

For test case:

  - input: เมื่อ 12 มี.ค. นายแอนโธนี เทรนกา ผู้พิพากษาศาลกลางในเมืองอเลกซานเดรีย รัฐเวอร์จิเนีย ในสหรัฐฯ สั่งให้เชลซี แมนนิง อดีตนักวิเคราะห์ข่าวกรองของกองทัพสหรัฐฯที่กลายเป็นสตรีข้ามเพศ ผู้ส่งต่อข้อมูลที่เป็นความลับของกองทัพสหรัฐฯไปให้กับเว็บไซต์วิกิลีกส์ ควรได้รับการปล่อยตัวจากเรือนจำ ซึ่งถูกกักขังตั้งแต่เดือน พ.ค.ปีกลายจากกรณีที่ไม่ให้ความร่วมมือกับคณะลูกขุนใหญ่ในการไต่สวนคดีวิกิลีกส์ทันที หลังคณะลูกขุนใหญ่สิ้นสุดการพิจารณานอกจากนี้ นายเทรนกายังปฏิเสธคำร้องของแมนนิงที่ให้ยกเลิกการจ่ายค่าปรับที่ไม่ยอมไปให้การในชั้นศาล ซึ่งเป็นเงินค่าปรับจำนวนทั้งสิ้น 256000 เหรียญสหรัฐฯ หรือราว 8 ล้านบาท ส่วนการพิจารณาฝากขังที่มีก

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✨ You're running DeepEval's latest Summarization Metric! (using gemini-2.0-flash, strict=False, 
async_mode=True)...

...



Metrics Summary

  - ✅ Summarization (score: 0.6, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash, reason: The score is 0.60 because the summary fails to address several questions that the original text answers, indicating a loss of key information., error: None)

For test case:

  - input: เมื่อวันที่ 28 มกราคม 2559 ผู้สื่อข่าวรายงานว่า หลังจากที่กระแสตุ๊กตาลูกเทพกำลังมาแรง นอกจากสร้างความเชื่อมั่นกับผู้ที่นิยมแล้ว ยังสร้างความหนักใจให้กับฝ่ายความมั่นคง เพราะล่าสุดถูกนำไปซุกซ่อนยาเสพติดจนกลายเป็นข่าวดังไปถึงประเทศเมียนมา,นายโก่ โก่อู ประธานสื่อมวลชนจังหวัดเมียวดี ประเทศเมียนมา ได้สอบถามมายังผู้สื่อข่าวฝั่งจังหวัดตาก ขอรายละเอียดเกี่ยวกับเรื่องนี้ เนื่องจากสื่อโทรทัศน์ในจังหวัดเมียวดีได้เสนอข่าวนี้ด้วย โดยล่าสุดนายโก่ โก่อู ได้แจ้งมาว่าทางด่านความมั่นคงฝั่งเมียวดี ซึ่งอยู่ที่สะพานมิตรภาพไทย-เมียนมา สั่งเจ้าหน้าที่ด่านตรวจเข้มงวดเรื่องตุ๊กตาทุกชนิด ไม่เว้นแม้แต่ตุ๊กตาลูกเทพด้วย หากมีการจะนำเข้า-ออก ให้มีการนำไปเอกซเรย์ตรวจสิ่งเสพติดทุกตัว,ขณะที่ ด้านด่านชายแดนถาวรฝั่

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✨ You're running DeepEval's latest Summarization Metric! (using gemini-2.0-flash, strict=False, 
async_mode=True)...

Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:02, ?test case/s]

Error evaluating pair 10: 'NoneType' object has no attribute 'truths'


In [7]:
# scores = []
# for r in results:
#     try:
#         scores.append(r.test_results[0].metrics_data[0].score)
#     except:
#         scores.append(0)

In [8]:
# scores = []
# for r in results:
#     try:
#         if r is None:
#             raise ValueError("Result is None")
#         scores.append(r.metrics_data[0].score)
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         scores.append(0)

In [9]:
df2 = pd.DataFrame({'body': df['body'], 'score': scores})
df2.to_csv('score-gemini.csv',index=False)